In [ ]:
import pandas as pd
path="/mnt/e/env/ts/datas/data/data_long/ft_normal/bingo5/by_unique_id/N1.csv"
df = pd.read_csv(path)  
df.columns.tolist()

In [ ]:
futr_cols

In [ ]:
# ================================================
# NeuralForecast Auto(TFT / PatchTST) フル実装（seed修正）
# ================================================
import os, json, time, warnings, inspect, random
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import pytorch_lightning as pl

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoTFT, AutoPatchTST
from neuralforecast.losses.pytorch import SMAPE

# ---------------------------
# ユーザ指定パラメータ
# ---------------------------
DATA_CSV = "/mnt/e/env/ts/datas/data/data_long/ft_normal/bingo5/by_unique_id/N1.csv"
TRIALS   = 1
SEED     = 1029
H        = 1
ARTIFACTS_ROOT = "/mnt/e/env/ts/ts-mlops-skeleton/nf_auto_runs"
FREQ = "D"
AUTO_GENERATE_FUTR_EXOG = False

# --- 再現性（Lightning/torch/np/python を一括固定）---
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
pl.seed_everything(SEED, workers=True)

# =========================================================
# ユーティリティ
# =========================================================
def ensure_dir(path: str) -> str:
    os.makedirs(path, exist_ok=True); return path

def now_str():
    return time.strftime("%Y%m%d-%H%M%S")

def to_datetime_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    out = df.copy(); out[col] = pd.to_datetime(out[col], errors="coerce"); return out

def to_numeric_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    out = df.copy(); out[col] = pd.to_numeric(out[col], errors="coerce"); return out

def preprocess_df(raw: pd.DataFrame) -> pd.DataFrame:
    df = raw.copy()
    if "unique_id" not in df.columns:
        df["unique_id"] = "series_0"
    df = to_datetime_col(df, "ds")
    df = to_numeric_col(df, "y")
    before = len(df)
    df = df.dropna(subset=["ds", "y"])
    dropped = before - len(df)
    if dropped > 0:
        print(f"[info] 前処理: ds/y の変換で {dropped} 行を除去しました。")
    df = df.sort_values(["unique_id", "ds"])
    df = df.drop_duplicates(subset=["unique_id", "ds"], keep="last").reset_index(drop=True)
    gap = df.groupby("unique_id")["ds"].diff().dropna()
    if not gap.empty and (gap != pd.Timedelta(days=1)).any():
        warnings.warn("日次ギャップが検出されました。")
    return df

def split_exog_by_prefix(df: pd.DataFrame):
    futr_cols = [c for c in df.columns if c.startswith("futr_")]
    hist_cols = [c for c in df.columns if c.startswith("hist_")]
    stat_cols = [c for c in df.columns if c.startswith("stat_")]
    return futr_cols, hist_cols, stat_cols

# ---- 指標 ----
def _to_arr(x): return np.asarray(x, dtype=float)
def smape(y, yhat, eps=1e-8): y=_to_arr(y);yhat=_to_arr(yhat);return 100*np.mean(2*np.abs(yhat-y)/(np.abs(y)+np.abs(yhat)+eps))
def mae(y, yhat): y=_to_arr(y);yhat=_to_arr(yhat);return float(np.mean(np.abs(yhat-y)))
def mape(y, yhat, eps=1e-8): y=_to_arr(y);yhat=_to_arr(yhat);den=np.clip(np.abs(y),eps,None);return 100*np.mean(np.abs((yhat-y)/den))
def rmse(y, yhat): y=_to_arr(y);yhat=_to_arr(yhat);return float(np.sqrt(np.mean((yhat-y)**2)))

def compute_metrics(df_pred: pd.DataFrame, model_cols):
    rows=[]
    for m in model_cols:
        y=df_pred["y"].values; yhat=df_pred[m].values
        rows.append({"model":m,"SMAPE":smape(y,yhat),"MAE":mae(y,yhat),"MAPE":mape(y,yhat),"RMSE":rmse(y,yhat),"n":len(df_pred)})
    return pd.DataFrame(rows)

def plot_last_window(cv_df: pd.DataFrame, model_cols, out_png: str, uid: str = None):
    if "cutoff" not in cv_df.columns:
        print("[warn] cutoff がないため描画スキップ"); return
    last_cutoff = cv_df["cutoff"].max()
    sub = cv_df[cv_df["cutoff"] == last_cutoff]
    if uid is not None:
        pick = sub[sub["unique_id"] == uid]
        if not pick.empty: sub = pick
    sub = sub.sort_values(["unique_id","ds"])
    if sub["unique_id"].nunique() > 1:
        sub = sub[sub["unique_id"] == sub["unique_id"].iloc[0]]
    plt.figure(figsize=(12,5))
    plt.plot(sub["ds"], sub["y"], label="y", linewidth=2)
    for m in model_cols: plt.plot(sub["ds"], sub[m], label=m, linewidth=1)
    plt.title(f"Last CV Window @ cutoff={last_cutoff}")
    plt.xlabel("ds"); plt.ylabel("value"); plt.legend()
    plt.tight_layout(); plt.savefig(out_png); plt.close()
    print(f"[info] 予測可視化を保存: {out_png}")

# =========================================================
# 1) データ読込 & 前処理
# =========================================================
print(f"[info] CSV を読込中: {DATA_CSV}")
raw = pd.read_csv(DATA_CSV)
print(f"[info] loaded shape: {raw.shape}")
print(f"[info] columns: {list(raw.columns)[:10]} ...")

df = preprocess_df(raw)
futr_cols, hist_cols, stat_cols = split_exog_by_prefix(df)
print(f"[info] exog sizes (raw): futr={len(futr_cols)} hist={len(hist_cols)} stat={len(stat_cols)}")

# =========================================================
# 2) static_df 構築（非数値→factorize）＆ df から stat_* を削除
# =========================================================
static_df = None
if len(stat_cols):
    static_df = df[["unique_id", *stat_cols]].drop_duplicates()
    for c in stat_cols:
        if not np.issubdtype(static_df[c].dtype, np.number):
            codes, uniques = pd.factorize(static_df[c], sort=True)
            static_df[c] = codes.astype(np.int32)
            print(f"[info] static_df: 非数値列をコード化 -> {c} （{len(uniques)}カテゴリ）")
    df = df.drop(columns=stat_cols)
    print(f"[info] df から stat_* を削除: {len(stat_cols)} 列")

# 最新の futr/hist を df から再抽出（stat は df に存在しない）
futr_cols, hist_cols, _ = split_exog_by_prefix(df)
print(f"[info] exog sizes (after drop stat): futr={len(futr_cols)} hist={len(hist_cols)} stat={len(stat_cols)}")

# =========================================================
# 3) futr_/hist_ の非数値対策（非数値は強制数値化 or 全欠損ならドロップ）
# =========================================================
protected = {"unique_id","ds","y"}
temporal_cols = [c for c in df.columns if c not in protected]
drop_cols=[]
for c in temporal_cols:
    if not np.issubdtype(df[c].dtype, np.number):
        coerced = pd.to_numeric(df[c], errors="coerce")
        if coerced.isna().all():
            drop_cols.append(c)
        else:
            df[c] = coerced.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
            print(f"[info] temporal: {c} を数値化（欠損は前後詰め→0）")
if drop_cols:
    df = df.drop(columns=drop_cols)
    print(f"[info] temporal: 数値化不能のためドロップ -> {drop_cols}")

# 存在確認で最終リスト
futr_cols = [c for c in futr_cols if c in df.columns]
hist_cols = [c for c in hist_cols if c in df.columns]
print(f"[info] exog sizes (final): futr={len(futr_cols)} hist={len(hist_cols)} stat={len(stat_cols)}")
print(f"[info] 学習に使う futr = {len(futr_cols)}, hist = {len(hist_cols)}, stat = {len(stat_cols)}")

# =========================================================
# 4) Auto モデル設定（config(trial) に外生を渡す）※ seedは入れない
# =========================================================
def tft_config(trial):
    return {
        "input_size":   trial.suggest_categorical("input_size",   [H*3, H*4, H*5]),
        "hidden_size":  trial.suggest_categorical("hidden_size",  [64, 128]),
        "dropout":      trial.suggest_float("dropout", 0.0, 0.2),
        "learning_rate":trial.suggest_float("learning_rate", 1e-4, 3e-3, log=True),
        "max_steps":    400,
        "futr_exog_list": futr_cols,
        "hist_exog_list": hist_cols,
        "stat_exog_list": stat_cols,  # static_df と対応
        # ここに "seed": SEED を入れないこと（Lightning Trainerに渡ってエラー化）
    }

def patchtst_config(trial):
    return {
        "input_size":   trial.suggest_categorical("input_size", [H*3, H*4, H*5]),
        "patch_len":    trial.suggest_categorical("patch_len",  [7, 14, 21, 28]),
        "stride":       trial.suggest_categorical("stride",     [1, 2, 4]),
        "n_layers":     trial.suggest_categorical("n_layers",   [2, 3]),
        "d_model":      trial.suggest_categorical("d_model",    [64, 128]),
        "n_heads":      trial.suggest_categorical("n_heads",    [2, 4]),
        "d_ff":         trial.suggest_categorical("d_ff",       [256, 512, 1024]),
        "dropout":      trial.suggest_float("dropout", 0.0, 0.2),
        "learning_rate":trial.suggest_float("learning_rate", 1e-4, 3e-3, log=True),
        "max_steps":    400,
        "futr_exog_list": futr_cols,
        "hist_exog_list": hist_cols,
        "stat_exog_list": stat_cols,
        # ここも seed を入れない
    }

# =========================================================
# 5) trials の渡し先を環境で自動切替
# =========================================================
fit_accepts_n_trials = "n_trials" in inspect.signature(NeuralForecast.fit).parameters

if fit_accepts_n_trials:
    models = [
        AutoTFT(h=H, loss=SMAPE(), backend="optuna", config=tft_config, verbose=True),
        AutoPatchTST(h=H, loss=SMAPE(), backend="optuna", config=patchtst_config, verbose=True),
    ]
    nf = NeuralForecast(models=models, freq=FREQ)
    print(f"[info] 学習開始: n_trials={TRIALS}, val_size={H*2}, freq={FREQ}（fit に n_trials）")
    nf.fit(df=df, static_df=static_df, val_size=H*2, n_trials=TRIALS)
else:
    models = [
        AutoTFT(h=H, loss=SMAPE(), backend="optuna", config=tft_config, num_samples=TRIALS, verbose=True),
        AutoPatchTST(h=H, loss=SMAPE(), backend="optuna", config=patchtst_config, num_samples=TRIALS, verbose=True),
    ]
    nf = NeuralForecast(models=models, freq=FREQ)
    print(f"[info] 学習開始: num_samples={TRIALS}, val_size={H*2}, freq={FREQ}（__init__ に num_samples）")
    nf.fit(df=df, static_df=static_df, val_size=H*2)

print("[info] 学習完了")

# =========================================================
# 6) 成果物保存
# =========================================================
run_dir   = ensure_dir(os.path.join(ARTIFACTS_ROOT, f"run_{now_str()}"))
models_dir= ensure_dir(os.path.join(run_dir, "models"))
plots_dir = ensure_dir(os.path.join(run_dir, "plots"))
tables_dir= ensure_dir(os.path.join(run_dir, "tables"))

meta = {
    "DATA_CSV": DATA_CSV, "TRIALS": TRIALS, "SEED": SEED, "H": H, "FREQ": FREQ,
    "futr_exog_list": futr_cols, "hist_exog_list": hist_cols, "stat_exog_list": stat_cols,
    "models": [m.__class__.__name__ for m in models],
    "fit_accepts_n_trials": fit_accepts_n_trials,
}
with open(os.path.join(run_dir, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

nf.save(models_dir)
print(f"[info] モデルを保存しました: {models_dir}")

# =========================================================
# 7) ロード確認
# =========================================================
nf_loaded = NeuralForecast.load(models_dir)
print("[info] モデルをロードしました。")

# =========================================================
# 8) CV → 指標・可視化
# =========================================================
print("[info] 交差検証（Rolling Origin）を実行します...")
try:
    cv_df = nf_loaded.cross_validation(df=df, static_df=static_df, n_windows=3, step_size=H, h=H)
except TypeError:
    cv_df = nf_loaded.cross_validation(df=df, static_df=static_df, n_windows=3, step_size=H, max_horizon=H)

fixed_cols = {"unique_id","ds","y","cutoff"}
model_cols = [c for c in cv_df.columns if c not in fixed_cols]
cv_path = os.path.join(tables_dir, "cv_predictions.csv")
cv_df.to_csv(cv_path, index=False); print(f"[info] CV 予測テーブルを保存: {cv_path}")

metrics_df = compute_metrics(cv_df, model_cols)
metrics_path = os.path.join(tables_dir, "metrics.csv")
metrics_df.to_csv(metrics_path, index=False); print(f"[info] 評価指標を保存: {metrics_path}")
print(metrics_df)

plot_last_window(cv_df, model_cols, out_png=os.path.join(plots_dir, "last_window.png"))

# =========================================================
# 9) 将来予測（任意）
# =========================================================
if AUTO_GENERATE_FUTR_EXOG:
    pass

print(f"[done] 成果物ルート: {run_dir}")


In [ ]:
# ================================================
# NeuralForecast Auto(TFT / PatchTST) 最小設定版
# ================================================
import os, json, time, warnings, inspect, random, math
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- OOM 断片化対策（torch import 前に）----
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

import torch
import pytorch_lightning as pl

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoTFT, AutoPatchTST
from neuralforecast.losses.pytorch import SMAPE

# ---------------------------
# ユーザ指定パラメータ
# ---------------------------
DATA_CSV = "/mnt/e/env/ts/datas/data/data_long/ft_normal/bingo5/by_unique_id/N1.csv"
TRIALS   = 16
SEED     = 1029
H        = 28
ARTIFACTS_ROOT = "/mnt/e/env/ts/ts-mlops-skeleton/nf_auto_runs"
FREQ = "D"

# ---------------------------
# Auto設定パラメータ（最小構成）
# ---------------------------
LOSS = SMAPE()
VALID_LOSS = None
SEARCH_ALG = "TPESampler"  # Optuna: TPESampler, Ray: BasicVariantGenerator
BACKEND = "optuna"
CALLBACKS = None
LOCAL_SCALER_TYPE = "standard"
EARLY_STOP_PATIENCE_STEPS = 20  # 適切な早期停止（20ステップで改善なければ停止）
VERBOSE = True

# リソース設定
CPUS = 4
GPUS = 1 if torch.cuda.is_available() else 0

# 外生変数の最大数（省メモリ）
TOPK_HIST = 32
TOPK_FUTR = 16

# ---- 再現性固定 ----
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
pl.seed_everything(SEED, workers=True)

# ---- 便利関数 ----
def ensure_dir(path: str) -> str:
    os.makedirs(path, exist_ok=True); return path

def now_str():
    return time.strftime("%Y%m%d-%H%M%S")

def to_datetime_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    out = df.copy(); out[col] = pd.to_datetime(out[col], errors="coerce"); return out

def to_numeric_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    out = df.copy(); out[col] = pd.to_numeric(out[col], errors="coerce"); return out

def preprocess_df(raw: pd.DataFrame) -> pd.DataFrame:
    df = raw.copy()
    if "unique_id" not in df.columns:
        df["unique_id"] = "series_0"
    df = to_datetime_col(df, "ds")
    df = to_numeric_col(df, "y")
    before = len(df)
    df = df.dropna(subset=["ds", "y"])
    if before - len(df) > 0:
        print(f"[info] 前処理: ds/y の変換で {before-len(df)} 行を除去しました。")
    df = df.sort_values(["unique_id", "ds"]).drop_duplicates(subset=["unique_id","ds"], keep="last")
    df = df.reset_index(drop=True)
    gap = df.groupby("unique_id")["ds"].diff().dropna()
    if not gap.empty and (gap != pd.Timedelta(days=1)).any():
        warnings.warn("日次ギャップが検出されました。")
    return df

def split_exog_by_prefix(df: pd.DataFrame):
    futr_cols = [c for c in df.columns if c.startswith("futr_")]
    hist_cols = [c for c in df.columns if c.startswith("hist_")]
    stat_cols = [c for c in df.columns if c.startswith("stat_")]
    return futr_cols, hist_cols, stat_cols

def factorize_static(static_df: pd.DataFrame) -> pd.DataFrame:
    out = static_df.copy()
    for c in out.columns:
        if c == "unique_id": 
            continue
        if not np.issubdtype(out[c].dtype, np.number):
            codes, uniques = pd.factorize(out[c], sort=True)
            out[c] = codes.astype(np.int32)
            print(f"[info] static_df: 非数値列をコード化 -> {c} （{len(uniques)}カテゴリ）")
    return out

def coerce_temporal_numeric(df: pd.DataFrame, protected=("unique_id","ds","y")):
    temporal_cols = [c for c in df.columns if c not in protected]
    drop_cols=[]
    for c in temporal_cols:
        if not np.issubdtype(df[c].dtype, np.number):
            coerced = pd.to_numeric(df[c], errors="coerce")
            if coerced.isna().all():
                drop_cols.append(c)
            else:
                df[c] = coerced.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
                print(f"[info] temporal: {c} を数値化（欠損は前後詰め→0）")
    if drop_cols:
        df.drop(columns=drop_cols, inplace=True)
        print(f"[info] temporal: 数値化不能のためドロップ -> {drop_cols}")
    return df

def select_topk_features(df: pd.DataFrame, feature_cols, k: int, target_col="y", min_std=1e-12):
    if k <= 0 or len(feature_cols) == 0:
        return []
    std = df[feature_cols].std(numeric_only=True)
    keep = std[std > min_std].index.tolist()
    if not keep:
        return []
    corrs = {}
    y = df[target_col].astype(float)
    for c in keep:
        x = df[c].astype(float)
        valid = x.notna() & y.notna()
        if valid.sum() < 3:
            continue
        xc = x[valid]; yc = y[valid]
        if xc.std() < min_std:
            continue
        corrs[c] = float(abs(np.corrcoef(xc, yc)[0,1]))
    if not corrs:
        return keep[:k]
    ranked = sorted(corrs.items(), key=lambda kv: kv[1], reverse=True)
    picked = [c for c,_ in ranked[:k]]
    if len(picked) < k:
        rest = [c for c in keep if c not in picked]
        picked += rest[:(k-len(picked))]
    return picked

def lightning_precision():
    try:
        major = int(pl.__version__.split(".")[0])
    except Exception:
        major = 2
    return "16-mixed" if major >= 2 else 16

def plot_last_window(cv_df: pd.DataFrame, model_cols, out_png: str, uid: str = None):
    if "cutoff" not in cv_df.columns:
        print("[warn] cutoff がないため描画スキップ"); return
    last_cutoff = cv_df["cutoff"].max()
    sub = cv_df[cv_df["cutoff"] == last_cutoff]
    if uid is not None:
        pick = sub[sub["unique_id"] == uid]
        if not pick.empty: sub = pick
    sub = sub.sort_values(["unique_id","ds"])
    if sub["unique_id"].nunique() > 1:
        sub = sub[sub["unique_id"] == sub["unique_id"].iloc[0]]
    plt.figure(figsize=(12,5))
    plt.plot(sub["ds"], sub["y"], label="y", linewidth=2)
    for m in model_cols:
        plt.plot(sub["ds"], sub[m], label=m, linewidth=1)
    plt.title(f"Last CV Window @ cutoff={last_cutoff}")
    plt.xlabel("ds"); plt.ylabel("value"); plt.legend()
    plt.tight_layout(); plt.savefig(out_png); plt.close()
    print(f"[info] 予測可視化を保存: {out_png}")

# =========================================================
# 1) データ読込 & 前処理
# =========================================================
print(f"[info] CSV を読込中: {DATA_CSV}")
raw = pd.read_csv(DATA_CSV)
print(f"[info] loaded shape: {raw.shape}")
print(f"[info] columns: {list(raw.columns)[:10]} ...")

df = preprocess_df(raw)
futr_cols, hist_cols, stat_cols = split_exog_by_prefix(df)
print(f"[info] exog sizes (raw): futr={len(futr_cols)} hist={len(hist_cols)} stat={len(stat_cols)}")

# =========================================================
# 2) static_df 構築（非数値→factorize）＆ df から stat_* を削除
# =========================================================
static_df = None
if len(stat_cols):
    static_df = df[["unique_id", *stat_cols]].drop_duplicates()
    static_df = factorize_static(static_df)
    df = df.drop(columns=stat_cols)
    print(f"[info] df から stat_* を削除: {len(stat_cols)} 列")

futr_cols, hist_cols, _ = split_exog_by_prefix(df)

# =========================================================
# 3) 時系列側の非数値→数値化
# =========================================================
df = coerce_temporal_numeric(df)

futr_cols = [c for c in futr_cols if c in df.columns]
hist_cols = [c for c in hist_cols if c in df.columns]
print(f"[info] exog sizes (final): futr={len(futr_cols)} hist={len(hist_cols)} stat={(0 if static_df is None else static_df.shape[1]-1)}")

# =========================================================
# 4) 外生 Top-K 選抜（省メモリ）
# =========================================================
hist_sel = select_topk_features(df, hist_cols, k=TOPK_HIST, target_col="y")
futr_sel = select_topk_features(df, futr_cols, k=TOPK_FUTR, target_col="y")
print(f"[info] 選抜: hist={len(hist_sel)}/{len(hist_cols)} → {len(hist_sel)} 列使用")
print(f"[info] 選抜: futr={len(futr_sel)}/{len(futr_cols)} → {len(futr_sel)} 列使用")

stat_sel = [] if static_df is None else [c for c in static_df.columns if c!="unique_id"]

# =========================================================
# 5) Auto モデル設定（最小構成）
# =========================================================
prec = lightning_precision()
trainer_kwargs = dict(
    accelerator="gpu" if GPUS > 0 else "cpu",
    devices=GPUS if GPUS > 0 else None,
    precision=prec,
    enable_checkpointing=False,
    logger=False,
    enable_progress_bar=VERBOSE,
)

# AutoModelの設定は最小限に（h以外）
def tft_config(trial):
    return {
        "loss": LOSS,
        "valid_loss": VALID_LOSS,
        "scaler_type": LOCAL_SCALER_TYPE,
        "early_stop_patience_steps": EARLY_STOP_PATIENCE_STEPS,
        "futr_exog_list": futr_sel,
        "hist_exog_list": hist_sel,
        "stat_exog_list": stat_sel,
        "trainer_kwargs": trainer_kwargs,
    }

def patchtst_config(trial):
    return {
        "loss": LOSS,
        "valid_loss": VALID_LOSS,
        "scaler_type": LOCAL_SCALER_TYPE,
        "early_stop_patience_steps": EARLY_STOP_PATIENCE_STEPS,
        "futr_exog_list": futr_sel,
        "hist_exog_list": hist_sel,
        "stat_exog_list": stat_sel,
        "trainer_kwargs": trainer_kwargs,
    }

# NF の fit に n_trials があるかどうかで分岐
fit_accepts_n_trials = "n_trials" in inspect.signature(NeuralForecast.fit).parameters

def build_nf(trials=TRIALS):
    if fit_accepts_n_trials:
        models = [
            AutoTFT(
                h=H,  # 必須引数
                backend=BACKEND,
                config=tft_config,
                search_alg=SEARCH_ALG,
                callbacks=CALLBACKS,
                verbose=VERBOSE
            ),
            AutoPatchTST(
                h=H,  # 必須引数
                backend=BACKEND,
                config=patchtst_config,
                search_alg=SEARCH_ALG,
                callbacks=CALLBACKS,
                verbose=VERBOSE
            ),
        ]
        nf = NeuralForecast(models=models, freq=FREQ)
        return nf, dict(n_trials=trials)
    else:
        models = [
            AutoTFT(
                h=H,  # 必須引数
                backend=BACKEND,
                config=tft_config,
                num_samples=trials,
                search_alg=SEARCH_ALG,
                callbacks=CALLBACKS,
                verbose=VERBOSE
            ),
            AutoPatchTST(
                h=H,  # 必須引数
                backend=BACKEND,
                config=patchtst_config,
                num_samples=trials,
                search_alg=SEARCH_ALG,
                callbacks=CALLBACKS,
                verbose=VERBOSE
            ),
        ]
        nf = NeuralForecast(models=models, freq=FREQ)
        return nf, {}

# =========================================================
# 6) 学習（OOM 時は自動フォールバック）
# =========================================================
def try_fit(nf, fit_kwargs):
    print(f"[info] 学習開始: trials={TRIALS}, val_size={H*2}, freq={FREQ}")
    print(f"[info] early_stop={EARLY_STOP_PATIENCE_STEPS}, scaler={LOCAL_SCALER_TYPE}")
    if fit_accepts_n_trials:
        nf.fit(df=df, static_df=static_df, val_size=H*2, **fit_kwargs)
    else:
        nf.fit(df=df, static_df=static_df, val_size=H*2)

nf, fit_kwargs = build_nf(TRIALS)
try:
    try_fit(nf, fit_kwargs)
except Exception as e:
    msg = str(e)
    if "CUDA out of memory" in msg or isinstance(e, torch.cuda.OutOfMemoryError):
        print("[warn] CUDA OOM 検出。縮小構成でリトライします...")
        torch.cuda.empty_cache()
        
        # フォールバック: 外生変数をさらに削減
        TOPK_HIST_FALLBACK = 16
        TOPK_FUTR_FALLBACK = 8
        hist_sel = select_topk_features(df, hist_cols, k=TOPK_HIST_FALLBACK, target_col="y")
        futr_sel = select_topk_features(df, futr_cols, k=TOPK_FUTR_FALLBACK, target_col="y")
        print(f"[info] フォールバック選抜: hist={len(hist_sel)} / futr={len(futr_sel)}")

        # モデル再構築（試行数も削減）
        nf, fit_kwargs = build_nf(max(4, TRIALS//2))
        try_fit(nf, fit_kwargs)
    else:
        raise

print("[info] 学習完了")

# =========================================================
# 7) 成果物保存
# =========================================================
run_dir   = ensure_dir(os.path.join(ARTIFACTS_ROOT, f"run_{now_str()}"))
models_dir= ensure_dir(os.path.join(run_dir, "models"))
plots_dir = ensure_dir(os.path.join(run_dir, "plots"))
tables_dir= ensure_dir(os.path.join(run_dir, "tables"))

meta = {
    "DATA_CSV": DATA_CSV,
    "TRIALS": TRIALS,
    "SEED": SEED,
    "H": H,
    "FREQ": FREQ,
    "BACKEND": BACKEND,
    "SEARCH_ALG": SEARCH_ALG,
    "LOSS": str(LOSS),
    "VALID_LOSS": str(VALID_LOSS),
    "LOCAL_SCALER_TYPE": LOCAL_SCALER_TYPE,
    "EARLY_STOP_PATIENCE_STEPS": EARLY_STOP_PATIENCE_STEPS,
    "CPUS": CPUS,
    "GPUS": GPUS,
    "futr_exog_list": futr_sel,
    "hist_exog_list": hist_sel,
    "stat_exog_list": stat_sel,
    "fit_accepts_n_trials": fit_accepts_n_trials,
    "precision": prec,
}
with open(os.path.join(run_dir, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

nf.save(models_dir)
print(f"[info] モデルを保存しました: {models_dir}")

# =========================================================
# 8) ロード確認
# =========================================================
nf_loaded = NeuralForecast.load(models_dir)
print("[info] モデルをロードしました。")

# =========================================================
# 9) 交差検証 → 指標・可視化
# =========================================================
print("[info] 交差検証（Rolling Origin）を実行します...")
try:
    cv_df = nf_loaded.cross_validation(df=df, static_df=static_df, n_windows=3, step_size=H, h=H)
except TypeError:
    cv_df = nf_loaded.cross_validation(df=df, static_df=static_df, n_windows=3, step_size=H, max_horizon=H)

fixed_cols = {"unique_id","ds","y","cutoff"}
model_cols = [c for c in cv_df.columns if c not in fixed_cols]
cv_path = os.path.join(tables_dir, "cv_predictions.csv")
cv_df.to_csv(cv_path, index=False)
print(f"[info] CV 予測テーブルを保存: {cv_path}")

# 簡易メトリクス
def _to_arr(x): return np.asarray(x, dtype=float)
def smape_np(y,yhat,eps=1e-8): y=_to_arr(y);yhat=_to_arr(yhat);return 100*np.mean(2*np.abs(yhat-y)/(np.abs(y)+np.abs(yhat)+eps))
def mae_np(y,yhat): y=_to_arr(y);yhat=_to_arr(yhat);return float(np.mean(np.abs(yhat-y)))
def mape_np(y,yhat,eps=1e-8): y=_to_arr(y);yhat=_to_arr(yhat);den=np.clip(np.abs(y),eps,None);return 100*np.mean(np.abs((yhat-y)/den))
def rmse_np(y,yhat): y=_to_arr(y);yhat=_to_arr(yhat);return float(np.sqrt(np.mean((yhat-y)**2)))

rows=[]
for m in model_cols:
    y=cv_df["y"].values; yhat=cv_df[m].values
    rows.append({
        "model":m,
        "SMAPE":smape_np(y,yhat),
        "MAE":mae_np(y,yhat),
        "MAPE":mape_np(y,yhat),
        "RMSE":rmse_np(y,yhat),
        "n":len(cv_df)
    })
metrics_df = pd.DataFrame(rows)
metrics_path = os.path.join(tables_dir, "metrics.csv")
metrics_df.to_csv(metrics_path, index=False)
print(f"[info] 評価指標を保存: {metrics_path}")
print(metrics_df)

plot_last_window(cv_df, model_cols, out_png=os.path.join(plots_dir, "last_window.png"))

print(f"[done] 成果物ルート: {run_dir}")

In [1]:
# NF Auto — 汎用EXOG適応＆署名フィルタ改修版（CVでval_size明示＋ES一時無効フォールバック付き）
# - モデル能力に応じて F/H/S を自動で付与/除去
# - モデル __init__ 署名＋Trainer 署名で未知引数を除去（安全）
# - 主要パラの簡易シノニム変換でバージョン差に耐性
# - 交差検証でも val_size を明示、必要時は EarlyStopping を一時無効化

import os, json, time, warnings, random, inspect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

import torch
import pytorch_lightning as pl
import optuna

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoTFT, AutoPatchTST
from neuralforecast.models import TFT, PatchTST
from neuralforecast.losses.pytorch import SMAPE

# =============================
# 0) 実行パラメータ
# =============================
path= "/mnt/e/env/ts/datas/data/data_long/ft_normal/bingo5/by_unique_id/N1.csv"

DATA_CSV = os.environ.get("NF_DATA_CSV", path)
TRIALS   = int(os.environ.get("NF_TRIAL_NUM_SAMPLES", 1))
SEED     = int(os.environ.get("NF_SEED", 1029))
H        = int(os.environ.get("NF_H", 1))
ARTIFACTS_ROOT = os.environ.get("NF_ARTIFACTS_ROOT", "nf_auto_runs")
FREQ = os.environ.get("NF_FREQ", "D")

LOSS = SMAPE()
BACKEND = "optuna"
SEARCH_ALG = optuna.samplers.TPESampler(seed=SEED)
EARLY_STOP_PATIENCE_STEPS = int(os.environ.get("NF_EARLY_STOP", 2))
VERBOSE = True
TOPK_HIST = int(os.environ.get("NF_TOPK_HIST", 32))
TOPK_FUTR = int(os.environ.get("NF_TOPK_FUTR", 16))

CPUS = -1
GPUS = -1

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); pl.seed_everything(SEED, workers=True)
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# =============================
# 1) 前処理ユーティリティ
# =============================

def ensure_dir(p): os.makedirs(p, exist_ok=True); return p

def now_str(): return time.strftime("%Y%m%d-%H%M%S")

def to_datetime_col(df, col): out=df.copy(); out[col]=pd.to_datetime(out[col], errors="coerce"); return out

def to_numeric_col(df, col): out=df.copy(); out[col]=pd.to_numeric(out[col], errors="coerce"); return out


def preprocess_df(raw: pd.DataFrame) -> pd.DataFrame:
    df = raw.copy()
    if "unique_id" not in df.columns: df["unique_id"]="series_0"
    df = to_datetime_col(df, "ds"); df = to_numeric_col(df, "y")
    before=len(df); df=df.dropna(subset=["ds","y"]) 
    if before-len(df)>0: print(f("[info] 前処理: ds/y の変換で {before-len(df)} 行を除去しました。"))
    df=df.sort_values(["unique_id","ds"]).drop_duplicates(subset=["unique_id","ds"], keep="last").reset_index(drop=True)
    gap=df.groupby("unique_id")["ds"].diff().dropna()
    if not gap.empty:
        if (gap.dt.days.fillna(0)!=1).any() and FREQ=="D":
            warnings.warn("等間隔でないタイムスタンプが検出されました。freq 指定または欠損補完を確認してください。")
    return df


def split_exog_by_prefix(df):
    futr=[c for c in df.columns if c.startswith("futr_")]
    hist=[c for c in df.columns if c.startswith("hist_")]
    stat=[c for c in df.columns if c.startswith("stat_")]
    return futr,hist,stat


def factorize_static(static_df: pd.DataFrame) -> pd.DataFrame:
    out=static_df.copy()
    for c in out.columns:
        if c=="unique_id": continue
        if not np.issubdtype(out[c].dtype, np.number):
            codes, _ = pd.factorize(out[c], sort=True)
            out[c]=codes.astype(np.int32)
            print(f"[info] static_df: 非数値列をコード化 -> {c}")
    return out


def coerce_temporal_numeric(df: pd.DataFrame, protected=("unique_id","ds","y")):
    temporal=[c for c in df.columns if c not in protected]
    drop=[]
    for c in temporal:
        if not np.issubdtype(df[c].dtype, np.number):
            coerced=pd.to_numeric(df[c], errors="coerce")
            if coerced.isna().all():
                drop.append(c)
            else:
                df[c]=coerced.fillna(method="ffill").fillna(method="bfill").fillna(0.0)
                print(f"[info] temporal: {c} を数値化（欠損は前後詰め→0）")
    if drop:
        df.drop(columns=drop, inplace=True)
        print(f"[info] temporal: 数値化不能のためドロップ -> {drop}")
    return df


def select_topk_features(df, feature_cols, k: int, target_col="y", min_std=1e-12):
    if k<=0 or len(feature_cols)==0: return []
    std=df[feature_cols].std(numeric_only=True)
    keep=std[std>min_std].index.tolist()
    if not keep: return []
    corrs={}; y=df[target_col].astype(float)
    for c in keep:
        x=df[c].astype(float); valid=x.notna() & y.notna()
        if valid.sum()<3: continue
        xc=x[valid]; yc=y[valid]
        if xc.std()<min_std: continue
        corrs[c]=float(abs(np.corrcoef(xc,yc)[0,1]))
    if not corrs: return keep[:k]
    ranked=sorted(corrs.items(), key=lambda kv: kv[1], reverse=True)
    picked=[c for c,_ in ranked[:k]]
    if len(picked)<k:
        rest=[c for c in keep if c not in picked]; picked+=rest[:(k-len(picked))]
    return picked


def lightning_precision():
    try: major=int(pl.__version__.split(".")[0])
    except: major=2
    return "16-mixed" if major>=2 else 16

# =============================
# 2) モデル能力→EXOG 自動適応
# =============================

MODEL_EXOG_FALLBACK = {
    "Autoformer": dict(F=True, H=False, S=False),
    "BiTCN": dict(F=True, H=True, S=True),
    "DeepAR": dict(F=True, H=False, S=True),
    "DeepNPTS": dict(F=True, H=True, S=True),
    "DilatedRNN": dict(F=True, H=True, S=True),
    "FEDformer": dict(F=True, H=False, S=False),
    "GRU": dict(F=True, H=True, S=True),
    "HINT": dict(F=True, H=True, S=True),
    "Informer": dict(F=True, H=False, S=False),
    "iTransformer": dict(F=False, H=False, S=False),
    "KAN": dict(F=True, H=True, S=True),
    "LSTM": dict(F=True, H=True, S=True),
    "MLP": dict(F=True, H=True, S=True),
    "MLPMultivariate": dict(F=True, H=True, S=True),
    "NBEATS": dict(F=False, H=False, S=False),
    "NBEATSx": dict(F=True, H=True, S=True),
    "NHITS": dict(F=True, H=True, S=True),
    "NLinear": dict(F=False, H=False, S=False),
    "PatchTST": dict(F=False, H=False, S=False),
    "RMoK": dict(F=False, H=False, S=False),
    "RNN": dict(F=True, H=True, S=True),
    "SOFTS": dict(F=False, H=False, S=False),
    "StemGNN": dict(F=False, H=False, S=False),
    "TCN": dict(F=True, H=True, S=True),
    "TFT": dict(F=True, H=True, S=True),
    "TiDE": dict(F=True, H=True, S=True),
    "TimeMixer": dict(F=False, H=False, S=False),
    "TimeLLM": dict(F=False, H=False, S=False),
    "TimesNet": dict(F=True, H=False, S=False),
    "TimeXer": dict(F=True, H=False, S=False),
    "TSMixer": dict(F=False, H=False, S=False),
    "TSMixerx": dict(F=True, H=True, S=True),
    "VanillaTransformer": dict(F=True, H=False, S=False),
}


def exog_capabilities(model_cls, fallback_key: str):
    f = bool(getattr(model_cls, "EXOGENOUS_FUTR", MODEL_EXOG_FALLBACK.get(fallback_key, {}).get("F", False)))
    h = bool(getattr(model_cls, "EXOGENOUS_HIST", MODEL_EXOG_FALLBACK.get(fallback_key, {}).get("H", False)))
    s = bool(getattr(model_cls, "EXOGENOUS_STAT", MODEL_EXOG_FALLBACK.get(fallback_key, {}).get("S", False)))
    return dict(F=f, H=h, S=s)


def attach_exog(config: dict, model_cls, fallback_key: str, futr_sel, hist_sel, stat_sel):
    caps = exog_capabilities(model_cls, fallback_key)
    config.update({
        "futr_exog_list": futr_sel if caps["F"] else [],
        "hist_exog_list": hist_sel if caps["H"] else [],
        "stat_exog_list": stat_sel if caps["S"] else [],
    })
    return config, caps

# --- 署名フィルタとシノニム補正 ---

TRAINER_ALLOWED = (set(inspect.signature(pl.Trainer.__init__).parameters.keys()) - {"self"}) if hasattr(pl, "Trainer") else set()
ALWAYS_ALLOWED = {"futr_exog_list","hist_exog_list","stat_exog_list"}

def _apply_param_synonyms(cfg: dict, model_cls) -> dict:
    try:
        params = set(inspect.signature(model_cls.__init__).parameters.keys()) - {"self"}
    except Exception:
        params = set()

    out = dict(cfg)

    def rename(src, dst):
        if src in out and src not in params and dst in params:
            out[dst] = out.pop(src)

    rename("n_heads", "n_head")
    rename("n_head", "n_heads")
    rename("d_model", "d")
    rename("d_ff", "ff_dim")
    rename("learning_rate", "lr")
    return out


def filter_kwargs_by_signature(config: dict, model_cls):
    try:
        model_params = set(inspect.signature(model_cls.__init__).parameters.keys())
    except Exception:
        model_params = set()
    model_params.discard("self")

    cfg = _apply_param_synonyms(config, model_cls)

    allowed = model_params | TRAINER_ALLOWED | ALWAYS_ALLOWED
    filtered = {k: v for k, v in cfg.items() if k in allowed}
    return filtered


# =============================
# 3) データ読み込みと外生選抜
# =============================
print(f"[info] CSV を読込中: {DATA_CSV}")
raw = pd.read_csv(DATA_CSV)
print(f"[info] loaded shape: {raw.shape}")

df = preprocess_df(raw)
futr_cols, hist_cols, stat_cols = split_exog_by_prefix(df)
print(f"[info] exog sizes (raw): futr={len(futr_cols)} hist={len(hist_cols)} stat={len(stat_cols)}")

static_df=None
if len(stat_cols):
    static_df = df[["unique_id", *stat_cols]].drop_duplicates()
    static_df = factorize_static(static_df)
    df = df.drop(columns=stat_cols)

# 型整備
futr_cols, hist_cols, _ = split_exog_by_prefix(df)
df = coerce_temporal_numeric(df)

# 再確認
futr_cols = [c for c in futr_cols if c in df.columns]
hist_cols = [c for c in hist_cols if c in df.columns]
stat_sel = [] if static_df is None else [c for c in static_df.columns if c!="unique_id"]

# Top-K
hist_sel = select_topk_features(df, hist_cols, k=TOPK_HIST, target_col="y")
futr_sel = select_topk_features(df, futr_cols, k=TOPK_FUTR, target_col="y")
print(f"[info] 選抜: hist={len(hist_sel)}/{len(hist_cols)} | futr={len(futr_sel)}/{len(futr_cols)} | stat={len(stat_sel)}")

# =============================
# 4) Auto 構成（PL引数はフラットで渡す→内部で安全に抽出）
# =============================
prec = lightning_precision()
PL_FLAT = dict(
    accelerator="auto",
    devices="auto",
    precision=prec,
    enable_checkpointing=False,
    logger=False,
    enable_progress_bar=True,
)

def tft_config(trial: optuna.trial.Trial):
    cfg = {
        "input_size": trial.suggest_categorical("input_size", [2*H, 3*H, 4*H]),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 3e-3, log=True),
        "hidden_size": trial.suggest_categorical("hidden_size", [64, 128, 256]),
        "n_head": trial.suggest_categorical("n_head", [2, 4, 8]),
        "dropout": trial.suggest_float("dropout", 0.0, 0.3),
        "batch_size": trial.suggest_categorical("batch_size", [64, 128]),
        "max_steps": int(os.environ.get("NF_MAX_STEPS", 800)),
        "val_check_steps": int(os.environ.get("NF_VAL_CHECK_STEPS", 50)),
        "early_stop_patience_steps": EARLY_STOP_PATIENCE_STEPS,
        **PL_FLAT,
    }
    cfg, caps = attach_exog(cfg, TFT, "TFT", futr_sel, hist_sel, stat_sel)
    cfg = filter_kwargs_by_signature(cfg, TFT)
    print(f"[cap] TFT exog -> F={caps['F']} H={caps['H']} S={caps['S']}")
    return cfg


def patchtst_config(trial: optuna.trial.Trial):
    cfg = {
        "input_size": trial.suggest_categorical("input_size", [2*H, 3*H, 4*H]),
        "d_model": trial.suggest_categorical("d_model", [64, 128, 192]),
        "n_heads": trial.suggest_categorical("n_heads", [2, 4, 8]),
        "d_ff": trial.suggest_categorical("d_ff", [128, 256, 512]),
        "patch_len": trial.suggest_categorical("patch_len", [8, 16, 32]),
        "stride": trial.suggest_categorical("stride", [8, 16]),
        "dropout": trial.suggest_float("dropout", 0.0, 0.2),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 3e-3, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [64, 128]),
        "max_steps": int(os.environ.get("NF_MAX_STEPS", 800)),
        "val_check_steps": int(os.environ.get("NF_VAL_CHECK_STEPS", 50)),
        "early_stop_patience_steps": EARLY_STOP_PATIENCE_STEPS,
        **PL_FLAT,
    }
    cfg, caps = attach_exog(cfg, PatchTST, "PatchTST", futr_sel, hist_sel, stat_sel)
    cfg = filter_kwargs_by_signature(cfg, PatchTST)
    print(f"[cap] PatchTST exog -> F={caps['F']} H={caps['H']} S={caps['S']}")
    return cfg


models = [
    AutoTFT(h=H, loss=LOSS, backend=BACKEND, config=tft_config, search_alg=SEARCH_ALG, num_samples=TRIALS, verbose=VERBOSE),
    AutoPatchTST(h=H, loss=LOSS, backend=BACKEND, config=patchtst_config, search_alg=SEARCH_ALG, num_samples=TRIALS, verbose=VERBOSE),
]

nf = NeuralForecast(models=models, freq=FREQ, local_scaler_type="standard")

# =============================
# 5) 学習
# =============================

def try_fit():
    val_size = max(2*H, H)  # 余裕を持たせる
    print(f"[info] 学習開始: trials={TRIALS}, val_size={val_size}, freq={FREQ}")
    nf.fit(df=df, static_df=static_df, val_size=val_size)

try:
    try_fit()
except Exception as e:
    msg=str(e)
    if "CUDA out of memory" in msg or isinstance(e, torch.cuda.OutOfMemoryError):
        print("[warn] CUDA OOM 検出。縮小構成でリトライします...")
        torch.cuda.empty_cache()
        hist_small = select_topk_features(df, hist_cols, k=max(8, TOPK_HIST//2), target_col="y")
        futr_small = select_topk_features(df, futr_cols, k=max(4, TOPK_FUTR//2), target_col="y")
        # 再バインド（能力を再評価）
        hist_sel, futr_sel = hist_small, futr_small
        try_fit()
    else:
        raise

print("[info] 学習完了")

# =============================
# 6) 成果物保存・CV・指標
# =============================
run_dir   = ensure_dir(os.path.join(ARTIFACTS_ROOT, f"run_{now_str()}"))
models_dir= ensure_dir(os.path.join(run_dir, "models"))
plots_dir = ensure_dir(os.path.join(run_dir, "plots"))
tables_dir= ensure_dir(os.path.join(run_dir, "tables"))

meta = {
    "DATA_CSV": DATA_CSV, "TRIALS": TRIALS, "SEED": SEED, "H": H, "FREQ": FREQ,
    "BACKEND": BACKEND, "SEARCH_ALG": type(SEARCH_ALG).__name__, "LOSS": str(LOSS),
    "LOCAL_SCALER_TYPE": "standard", "EARLY_STOP_PATIENCE_STEPS": EARLY_STOP_PATIENCE_STEPS,
    "CPUS": CPUS, "GPUS": GPUS,
    "futr_exog_list": futr_sel, "hist_exog_list": hist_sel, "stat_exog_list": stat_sel,
    "precision": prec,
}
with open(os.path.join(run_dir, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

nf.save(models_dir)
print(f"[info] モデルを保存しました: {models_dir}")

nf_loaded = NeuralForecast.load(models_dir)
print("[info] モデルをロードしました。")

print("[info] 交差検証（Rolling Origin）を実行します...")
val_size_cv = max(2*H, H)
try:
    # PL2 系では ES の監視名が 'ptl/val_loss' になる実装が多いが、
    # そもそも val loader が無いと生成されないため、val_size を必ず明示する
    cv_df = nf_loaded.cross_validation(
        df=df, static_df=static_df,
        n_windows=3, step_size=H, h=H,
        val_size=val_size_cv
    )
except RuntimeError as e:
    # EarlyStopping が監視指標未出力で落ちた場合のフォールバック：CV 中のみ ES を無効化して再実行
    if "Early stopping conditioned on metric" in str(e):
        print("[warn] CV中の EarlyStopping 監視メトリクス未検出。CVに限り EarlyStopping を一時無効化して再実行します。")
        for m in nf_loaded.models:
            if hasattr(m, "early_stop_patience_steps"):
                try:
                    m.early_stop_patience_steps = None
                except Exception:
                    pass
            if hasattr(m, "trainer_kwargs"):
                try:
                    m.trainer_kwargs.pop("early_stop_patience_steps", None)
                except Exception:
                    pass
        cv_df = nf_loaded.cross_validation(
            df=df, static_df=static_df,
            n_windows=3, step_size=H, h=H,
            val_size=val_size_cv
        )
    else:
        raise
except TypeError:
    # バージョン差：max_horizon 名称の互換
    try:
        cv_df = nf_loaded.cross_validation(
            df=df, static_df=static_df,
            n_windows=3, step_size=H, max_horizon=H,
            val_size=val_size_cv
        )
    except RuntimeError as e:
        if "Early stopping conditioned on metric" in str(e):
            print("[warn] CV中の EarlyStopping 監視メトリクス未検出。CVに限り EarlyStopping を一時無効化して再実行します。")
            for m in nf_loaded.models:
                if hasattr(m, "early_stop_patience_steps"):
                    try:
                        m.early_stop_patience_steps = None
                    except Exception:
                        pass
                if hasattr(m, "trainer_kwargs"):
                    try:
                        m.trainer_kwargs.pop("early_stop_patience_steps", None)
                    except Exception:
                        pass
            cv_df = nf_loaded.cross_validation(
                df=df, static_df=static_df,
                n_windows=3, step_size=H, max_horizon=H,
                val_size=val_size_cv
            )
        else:
            raise

fixed_cols = {"unique_id","ds","y","cutoff"}
model_cols = [c for c in cv_df.columns if c not in fixed_cols]
cv_path = os.path.join(tables_dir, "cv_predictions.csv")
cv_df.to_csv(cv_path, index=False)
print(f"[info] CV 予測テーブルを保存: {cv_path}")

# 指標
_def_to_arr = lambda x: np.asarray(x, dtype=float)

def smape_np(y,yhat,eps=1e-8): y=_def_to_arr(y);yhat=_def_to_arr(yhat);return 100*np.mean(2*np.abs(yhat-y)/(np.abs(y)+np.abs(yhat)+eps))

def mae_np(y,yhat): y=_def_to_arr(y);yhat=_def_to_arr(yhat);return float(np.mean(np.abs(yhat-y)))

def mape_np(y,yhat,eps=1e-8): y=_def_to_arr(y);yhat=_def_to_arr(yhat);den=np.clip(np.abs(y),eps,None);return 100*np.mean(np.abs((yhat-y)/den))

def rmse_np(y,yhat): y=_def_to_arr(y);yhat=_def_to_arr(yhat);return float(np.sqrt(np.mean((yhat-y)**2)))

rows=[]
for m in model_cols:
    y=cv_df["y"].values; yhat=cv_df[m].values
    rows.append({"model":m, "SMAPE":smape_np(y,yhat), "MAE":mae_np(y,yhat), "MAPE":mape_np(y,yhat), "RMSE":rmse_np(y,yhat), "n":len(cv_df)})
metrics_df = pd.DataFrame(rows)
metrics_path = os.path.join(tables_dir, "metrics.csv")
metrics_df.to_csv(metrics_path, index=False)
print(f"[info] 評価指標を保存: {metrics_path}\n{metrics_df}")

# 描画
if not cv_df.empty:
    try:
        last = cv_df["cutoff"].max()
        sub = cv_df[cv_df["cutoff"] == last]
        if sub["unique_id"].nunique() > 1:
            sub = sub[sub["unique_id"] == sub["unique_id"].iloc[0]]
        plt.figure(figsize=(12,5))
        plt.plot(sub["ds"], sub["y"], label="y", linewidth=2)
        for m in model_cols: plt.plot(sub["ds"], sub[m], label=m, linewidth=1)
        plt.title(f"Last CV Window @ cutoff={last}")
        plt.xlabel("ds"); plt.ylabel("value"); plt.legend(); plt.tight_layout()
        out_png = os.path.join(plots_dir, "last_window.png")
        plt.savefig(out_png); plt.close()
        print(f"[info] 予測可視化を保存: {out_png}")
    except Exception:
        print("[warn] 可視化に失敗しました（処理を継続します）。")

print(f"[done] 成果物ルート: {run_dir}")


Seed set to 1029
/home/az/miniconda3/envs/nc/lib/python3.11/site-packages/torch/__init__.py:1628: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:45.)
  _C._set_float32_matmul_precision(precision)
/tmp/ipykernel_365392/3762487492.py:75: UserWarning: 等間隔でないタイムスタンプが検出されました。freq 指定または欠損補完を確認してください。
  warnings.warn("等間隔でないタイムスタンプが検出されました。freq 指定または欠損補完を確認してください。")
[I 2025-11-12 07:52:04,089] A new study created in memory with name: no-name-eca17002-db25-4948-a0be-f2d3f1773b4b


[info] CSV を読込中: /mnt/e/env/ts/datas/data/data_long/ft_normal/bingo5/by_unique_id/N1.csv
[info] loaded shape: (364, 162)
[info] exog sizes (raw): futr=34 hist=122 stat=2
[info] static_df: 非数値列をコード化 -> stat_ds_quarteryear
[info] static_df: 非数値列をコード化 -> stat_ds_month_lbl
[info] 選抜: hist=32/122 | futr=16/34 | stat=2
[cap] TFT exog -> F=True H=True S=True
[cap] PatchTST exog -> F=False H=False S=False
[info] 学習開始: trials=1, val_size=2, freq=D


  0%|          | 0/1 [00:00<?, ?it/s]

Seed set to 1
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[cap] TFT exog -> F=True H=True S=True


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[info] CV 予測テーブルを保存: nf_auto_runs/run_20251112-075246/tables/cv_predictions.csv
[info] 評価指標を保存: nf_auto_runs/run_20251112-075246/tables/metrics.csv
          model      SMAPE       MAE       MAPE      RMSE  n
0       AutoTFT        NaN       NaN        NaN       NaN  3
1  AutoPatchTST  25.273531  1.123584  31.792946  1.388669  3
[info] 予測可視化を保存: nf_auto_runs/run_20251112-075246/plots/last_window.png
[done] 成果物ルート: nf_auto_runs/run_20251112-075246


In [5]:
# === モデルのプロパティ値を安全に一覧表示するユーティリティ ===
import inspect
import numpy as np
import pandas as pd

try:
    import torch
except Exception:
    torch = None  # torch が無い環境でも動くように

SIMPLE_TYPES = (str, int, float, bool, type(None))

def _summarize_value(v, maxlen=200):
    """値を安全・短く要約して表示用に整形"""
    try:
        # 素朴な型はそのまま（長すぎる文字列は詰める）
        if isinstance(v, SIMPLE_TYPES):
            s = str(v)
            return (s[:maxlen] + "…") if len(s) > maxlen else s

        # numpy
        if isinstance(v, np.ndarray):
            return f"np.ndarray(shape={v.shape}, dtype={v.dtype})"

        # pandas
        if isinstance(v, pd.DataFrame):
            return f"DataFrame(shape={v.shape}, columns={list(v.columns)[:6]}{'…' if v.shape[1]>6 else ''})"
        if isinstance(v, pd.Series):
            return f"Series(len={len(v)}, name={v.name}, dtype={v.dtype})"

        # torch
        if torch is not None:
            if isinstance(v, torch.Tensor):
                dev = v.device if hasattr(v, 'device') else 'cpu'
                return f"torch.Tensor(shape={tuple(v.shape)}, dtype={v.dtype}, device={dev})"
            if hasattr(v, "__class__") and v.__class__.__module__.startswith("torch.optim"):
                return f"{v.__class__.__name__}(param_groups={len(getattr(v,'param_groups',[]))})"

        # リスト／タプル／セット／辞書はサイズだけ
        if isinstance(v, (list, tuple, set)):
            return f"{type(v).__name__}(len={len(v)})"
        if isinstance(v, dict):
            keys = list(v.keys())[:8]
            return f"dict(len={len(v)}, keys={keys}{'…' if len(v)>8 else ''})"

        # その他のオブジェクトは型名＋一部repr
        r = repr(v)
        r = r.replace("\n", " ")
        if len(r) > maxlen:
            r = r[:maxlen] + "…"
        return f"{v.__class__.__name__}: {r}"
    except Exception as e:
        return f"<summarize_error: {e}>"

def collect_properties(obj, include_private=False):
    """
    obj の「公開属性（__dict__）＋ @property を含む」名前→値の辞書を作る。
    値の取得で例外が出ても飲み込んで続行。
    """
    seen = set()
    out = {}

    # 1) 実インスタンス属性
    for k, v in getattr(obj, "__dict__", {}).items():
        if not include_private and k.startswith("_"):
            continue
        seen.add(k)
        out[k] = v

    # 2) @property で定義された属性
    try:
        for name, member in inspect.getmembers(type(obj)):
            if not isinstance(member, property):
                continue
            if not include_private and name.startswith("_"):
                continue
            if name in seen:
                continue
            try:
                out[name] = getattr(obj, name)
            except Exception as e:
                out[name] = f"<property_error: {e}>"
    except Exception:
        pass

    # 3) dir で拾えるその他の公開属性（callable を除く）
    for name in dir(obj):
        if not include_private and name.startswith("_"):
            continue
        if name in seen:
            continue
        try:
            val = getattr(obj, name)
        except Exception as e:
            val = f"<attr_error: {e}>"
        # 関数やメソッドは除外
        if inspect.ismethod(val) or inspect.isfunction(val):
            continue
        out[name] = val

    return out

def print_properties(obj, title=None, sort=True, include_private=False):
    """プロパティを表形式でプリント（名前 / 型 / 要約）"""
    if title:
        print("="*len(title))
        print(title)
        print("="*len(title))
    props = collect_properties(obj, include_private=include_private)
    items = list(props.items())
    if sort:
        items.sort(key=lambda kv: kv[0])

    name_w = max(8, min(40, max((len(k) for k,_ in items), default=8)))
    type_w = 24

    header = f"{'name'.ljust(name_w)}  {'type'.ljust(type_w)}  value"
    print(header)
    print("-"*len(header))
    for k, v in items:
        t = type(v).__name__
        s = _summarize_value(v)
        print(f"{k.ljust(name_w)}  {t.ljust(type_w)}  {s}")

def print_nf_models_properties(nf):
    """
    NeuralForecast インスタンスから各モデルを辿って表示。
    Auto系モデルの場合は、チューニング後の実体（m.model）があればそれも併せて表示。
    """
    for i, m in enumerate(getattr(nf, "models", []), start=1):
        print_properties(m, title=f"[Model {i}] {m.__class__.__name__}")
        # AutoTFT/AutoPatchTST などは、最適化後に .model に実体が入る
        tuned = getattr(m, "model", None)
        if tuned is not None and tuned is not m:
            print_properties(tuned, title=f"[Model {i}] tuned -> {tuned.__class__.__name__}")

        # よく見る追加情報（存在すれば）
        trkw = getattr(m, "trainer_kwargs", None)
        if trkw is not None:
            print_properties(trkw, title=f"[Model {i}] trainer_kwargs (dict)")
        futr = getattr(m, "futr_exog_list", None)
        hist = getattr(m, "hist_exog_list", None)
        stat = getattr(m, "stat_exog_list", None)
        if any(x is not None for x in (futr, hist, stat)):
            print("\n[exog lists]")
            if futr is not None: print(f"  futr_exog_list: {futr}")
            if hist is not None: print(f"  hist_exog_list: {hist}")
            if stat is not None: print(f"  stat_exog_list: {stat}")
        print("\n")

# === 使い方（学習後 or ロード後）===
# 例: nf = NeuralForecast(models=[...], ...)
# nf.fit(...)

# 全モデルのプロパティ一覧を表示
print_nf_models_properties(nf)

# 単一モデルだけ見たい場合:
# print_properties(nf.models[0], title="First model")


[Model 1] AutoTFT
name                                      type                      value
-------------------------------------------------------------------------
CHECKPOINT_HYPER_PARAMS_KEY               str                       hyper_parameters
CHECKPOINT_HYPER_PARAMS_NAME              str                       hparams_name
CHECKPOINT_HYPER_PARAMS_TYPE              str                       hparams_type
EXOGENOUS_FUTR                            bool                      True
EXOGENOUS_HIST                            bool                      True
EXOGENOUS_STAT                            bool                      True
MULTIVARIATE                              bool                      False
RECURRENT                                 bool                      False
T_destination                             TypeVar                   TypeVar: ~T_destination
alias                                     NoneType                  None
allow_zero_length_dataloader_with_multiple_devices  boo